In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
! cp 'gdrive/My Drive/Ab_Summarization/Our_Own_Code/Seq2Seq Combos/Seq2Seq.py' .

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
import time
import shutil
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)

print('TensorFlow Version: {}'.format(tf.__version__))

from Seq2Seq import Seq2Seq_Model

TensorFlow Version: 1.13.1


**The different types of experiments**

<ol>
  <li> Type 1 - emb-pos, bahdanau, greedy, 2 enc-dec, 128 units </li>
  <li> Type 2 - emb-pos, luong, greedy, 2 enc-dec, 128 units </li>
  <li> Type 3 - emb-pos, bahdanau, beam(10), 2 enc-dec, 128 units </li>
  <li> Type 4 - emb-pos, luong, beam(10), 2 enc-dec, 128 units </li>
  <li> Type 5 - emb, bahdanau, greedy, 2 enc-dec, 128 units </li>
  <li> Type 6 - emb, luong, greedy, 2 enc-dec, 128 units </li>
  <li> Type 7 - emb, bahdanau, beam(10), 2 enc-dec, 128 units </li>
  <li> Type 8 - emb, luong, beam(10), 2 enc-dec, 128 units </li>
  <li> Type 9 - emb, None, greedy, 2 enc-dec, 128 units </li>
  <li> Type 10 - emb, None, beam(10), 2 enc-dec, 128 units </li>
  <li> Type 11 - emb-pos, None, greedy, 2 enc-dec, 128 units </li>
  <li> Type 12 - emb-pos, None, beam(10), 2 enc-dec, 128 units </li>
 </ol>

In [0]:
def pad_sentence_batch(sentence_batch, vocab_to_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

def get_batches(summaries, texts, batch_size, vocab_to_int):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch, vocab_to_int))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch, vocab_to_int))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [0]:
def full_model_run(type_val):
  
  path = 'gdrive/My Drive/Ab_Summarization/Our_Own_Code/sumdata/train/'
  ckpt_path = "gdrive/My Drive/Ab_Summarization/Our_Own_Code/Seq2Seq Combos/Model_Checkpoints_Final/"

  if 1 in [c in type_val for c in ['1','2','3','4','11','12']]:
    emb_name = 'word_embedding_matrix1'
  else:
    emb_name = 'word_embedding_matrix'

  if 1 in [c in type_val for c in ['1','3','5','7']]:
    attn = 'Bahdanau'
  else:
    attn = 'Luong'

  if 1 in [c in type_val for c in ['1','2','5','6','9','11']]:
    dec_mech = 'greedy'
  else:
    dec_mech = 'beam'

  if 1 in [c in type_val for c in ['9','10','11','12']]:
    attn = None
    
  sorted_summaries = pickle.load(open(path+'summaries.pkl', 'rb'))
  sorted_texts = pickle.load(open(path+'texts.pkl', 'rb'))
  word_embedding_matrix = pickle.load(open(path+emb_name, 'rb'))
  vocab_to_int = pickle.load(open(path+'vocab_to_int', 'rb'))
  int_to_vocab = pickle.load(open(path+'int_to_vocab', 'rb'))
  
  print("1. Loaded the data")
  
  params = {
      'attention_mechanism':attn, #Luong, Bahdanau, None
      'batch_size':128,
      'bidirectional':True, #False, True
      'cell_type':'LSTM',
      'embeddings':word_embedding_matrix,
      'vocab_size':word_embedding_matrix.shape[0],
      'inference_mechanism':dec_mech, #beam, greedy
      'num_decoder_layers':2,
      'num_encoder_layers':2,
      'num_units':128,
      'end_token':'<EOS>',
      'beam_size':10,
      'word2int':vocab_to_int
  }

  if not os.path.exists(os.path.join(ckpt_path,type_val)):
    os.makedirs(os.path.join(ckpt_path,type_val))
    print("2. Created new directory for this type")
  else:
    for f in os.listdir(os.path.join(ckpt_path,type_val)):
      shutil.copy2(os.path.join(ckpt_path,type_val,f),"./")
    print("2. Copied exisiting checkpoints")
  
  ob = Seq2Seq_Model(params)
  train_graph = ob.build_graph()
  
  print("3. Contructed the graph")
    
  # choose number of samples
  sorted_sum = sorted_summaries[:1000000]
  sorted_tex = sorted_texts[:1000000]
  del sorted_summaries, sorted_texts
  
  split = int(0.7*len(sorted_sum))
  train_summaries, test_summaries = sorted_sum[:300000], sorted_sum[split:]
  train_texts, test_texts = sorted_tex[:300000], sorted_tex[split:]
  del sorted_sum, sorted_tex
  
  # Train the Model
  learning_rate = 0.005
  learning_rate_decay = 0.95
  min_learning_rate = 0.0005
  stop_early = 0 
  batch_loss = []

  checkpoint = "final_best_model_multigraph.ckpt"

  with tf.Session(graph=train_graph) as sess:
    saver = tf.train.Saver()
    if os.path.isfile("./"+checkpoint+".index"):
      saver.restore(sess,"./"+checkpoint)

    else:
      sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter("./Logs/Seq2Seq", sess.graph)

    for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
        get_batches(train_summaries, train_texts, params['batch_size'], vocab_to_int)):
      start_time = time.time()
      _, loss = sess.run(
          [ob.train_op, ob.cost],
          {ob.encoder_inputs: texts_batch,
           ob.decoder_targets: summaries_batch,
           ob.learning_rate: learning_rate,
           ob.decoder_lengths: summaries_lengths,
           ob.encoder_lengths: texts_lengths})
      print("Batch No. %4d of %d Current Loss %.4f Time %0.3f"%(batch_i+1,int(len(train_summaries)/params['batch_size']),loss,time.time()-start_time))

      #Reduce learning rate, but not below its minimum value
      if(batch_i % 20 == 0):
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
          learning_rate = min_learning_rate

      # Saving for every few batches
      if batch_i % 500 == 0:
        print("New Checkpoint Created for this batch")
        saver.save(sess, "./"+checkpoint)

    saver.save(sess, "./"+checkpoint)
    
  print("4. Finished training")
  
  for f in os.listdir():
    if checkpoint.split(".")[0] in f or f == 'checkpoint':
      shutil.copy2("./"+f,os.path.join(ckpt_path,type_val))
      os.remove("./"+f)
      
  print("5. Copied checkpoints to drive")

In [0]:
# train all types for 1 epoch
for i in range(1,13):
  print("--------------------------------------------------------------------------------------------------------")
  type_val = "type"+str(i)
  print(type_val)
  full_model_run(type_val)
  print("--------------------------------------------------------------------------------------------------------")

In [0]:
# train any 1 particular type
type_val = 'type8'
full_model_run(type_val)